## 전처리 하기

#여러 문서

In [4]:
# 이 코드는 그냥 가져다 쓰자

from kiwipiepy import Kiwi
import re
from collections import Counter
kiwi = Kiwi()
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()
customized_stopwords = ['연합뉴스', '네이버', '기자', '문의']
for word in customized_stopwords:
    stopwords.add((word, 'NNG'))

def do_Kr_preprocessing(text):
    cleaned_content = re.sub(r'[^\w\d\s]','',text) # To remove symbols
    cleaned_content = cleaned_content.lower() # Case conversion, upper -> lower
    word_tokens = kiwi.tokenize(cleaned_content, stopwords=stopwords) 
    NN_words = []   # To select nouns
    for token in word_tokens:
        if 'NN' in token.tag:
            NN_words.append(token.form)
    final_NN_words = []
    for word in NN_words:
        if len(word) > 1:
            final_NN_words.append(word)
    
    return final_NN_words

In [8]:
# 네트워크 생성
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import networkx as nx
def construct_network(text):
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1))
    DTM_tf = vectorizer.fit_transform(text)
    feature_names = vectorizer.get_feature_names_out()
    DTM = np.array(DTM_tf.todense())
    DTM_binary = np.sign(DTM)
    words_cooccurrence = np.dot(DTM_binary.T, DTM_binary)
    np.fill_diagonal(words_cooccurrence, 0)
    #print(feature_names) # 네트워크를 구성하는 단어 출력하기
    #print(words_cooccurrence) # 단어들 간의 인접행렬 출력하기
    g = nx.convert_matrix.from_numpy_array(words_cooccurrence)
    mapping = {}
    for k, word in enumerate(feature_names):
        mapping[k]=word
    g1 = nx.relabel_nodes(g, mapping)
    return g1

def get_text_network(text, selected_words):        
    cleaned_docs = [do_Kr_preprocessing(doc) for doc in text]
    new_docs=[]
    for doc in cleaned_docs:
        new_doc=[]
        for word in doc:
            if word in selected_words:
                new_doc.append(word)
        if len(new_doc)>1:
            new_docs.append(' '.join(new_doc))
    return construct_network(new_docs)

### Text data 불러오기

In [12]:
import numpy as np
from os import listdir
from os.path import isfile, join

mypath = './example_Kr/' # 네트워크 분석을 하고자하는 문서들이 저장되어 있는 폴더
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles.sort()

total_docs = []
for file in onlyfiles:
    file_path = mypath+file
    with open(file_path, 'r', encoding='utf8') as f:
        content = f.read()
    total_docs.append(content)

In [14]:
total_docs

['사과 과일 여름 바나나 딸기.\n딸기 사과 바지 옷 하늘 구름.',
 '우리나라 대한민국 일본 중국 미국.\n대한민국 북한 일본 중국.',
 '대한민국 미국 축구 북한 농구.\n축구 북한 대한민국 야구.\n가을 축구 대한민국 손흥민.',
 '바나나 사과 주스 건강 매일.\n키위 바나나 주스 아침.\n건강 달리기 걷기 과일.',
 '가을 자전거 하늘 걷기.\n도로 주행 여행 드라이브.\n겨울 눈 추위 감기.']

In [16]:
final_nouns = ['사과', '바나나', '주스', '축구', '대한민국']

# Text network 생성하기

In [19]:
g = get_text_network(total_docs, final_nouns)

In [20]:
g['바나나']   #바나나와 사과가 함께 쓰인 문서는 2개이고, 바나나와 주스가 함께 쓰인 문서는 1개임을 알 수 있음

AtlasView({'사과': {'weight': 2}, '주스': {'weight': 1}})

In [23]:
# graphml 파일로 저장하기
nx.write_graphml(g, 'Kr_test_docs.graphml')

In [25]:
#한글과 영어 시각화 차이
# gephi에서 영어 폰트가 기본 선택임 -> 한글 폰트로 바꾸면 됨